For storage and sharing of network design and analysis information we will be using JSON (Javascrip Object Notation). This is a standardized file and transfer format and more information can be found at <http://www.json.org>.

Idea: Python can read and write JSON strings and files.  However that doesn't mean it understands what you intend to do with the JSON. Furthermore Python has many more and more powerful data structures than JSON supports (JSON is kept intentionally simple). Hence we will be pre-processing and post-processing in the JSON export and JSON import processes repectively to get the best features of both Python and JSON.

Some of the output panes have been minimized. Use a single click to expand them and a double click to minimize them again.

The first thing to do is import the json module:

In [1]:
import json

Consider the following hypothetical demand dictionary for a three node network:

In [2]:
demands = {(1, 2): 5, (1, 3): 7, (2, 1): 5, (2, 3): 8, (3, 1): 7, (3, 2): 8}

If we try to naively convert this to a JSON string we get the following error:

In [3]:
print json.dumps(demands)

TypeError: keys must be a string

Looking up the json.dumps function tells us:

In [4]:
help(json.dumps)

Help on function dumps in module json:

dumps(obj, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, encoding='utf-8', default=None, sort_keys=False, **kw)
    Serialize ``obj`` to a JSON formatted ``str``.
    
    If ``skipkeys`` is false then ``dict`` keys that are not basic types
    (``str``, ``unicode``, ``int``, ``long``, ``float``, ``bool``, ``None``)
    will be skipped instead of raising a ``TypeError``.
    
    If ``ensure_ascii`` is false, all non-ASCII characters are not escaped, and
    the return value may be a ``unicode`` instance. See ``dump`` for details.
    
    If ``check_circular`` is false, then the circular reference check
    for container types will be skipped and a circular reference will
    result in an ``OverflowError`` (or worse).
    
    If ``allow_nan`` is false, then it will be a ``ValueError`` to
    serialize out of range ``float`` values (``nan``, ``inf``, ``-inf``) in
    strict compli

*Although* it looks like the `default` function should be able to help us out with the conversion, this is not the case for a standard type like a dictionary (dict) since it ignores the `default` function and repeats the error.

An alternative approach is to just define a helper function that takes us a bit farther towards the final JSON format that we'd like without doing everything from scratch:

In [5]:
def demands_to_j(obj):
    """ A function to partially convert a demand dictionary to JSON format."""
    tmp = []
    for d in obj.keys():
        tmp.append({"source": d[0], "target": d[1], "demand": obj[d]})
    return tmp

Now we can obtain a nice JSON representation of our demand dictionary as so:

In [6]:
print json.dumps(demands_to_j(demands))

[{"source": 1, "target": 2, "demand": 5}, {"source": 3, "target": 2, "demand": 8}, {"source": 1, "target": 3, "demand": 7}, {"source": 3, "target": 1, "demand": 7}, {"source": 2, "target": 1, "demand": 5}, {"source": 2, "target": 3, "demand": 8}]


There are several keyword arguments we can feed to `json.dumps` to give us nicer human readable output. These include `sort_keys` and `indent`.


In [7]:
print json.dumps(demands_to_j(demands), sort_keys=True)
print "Now with indentation"
print json.dumps(demands_to_j(demands), sort_keys=True, indent=4)

[{"demand": 5, "source": 1, "target": 2}, {"demand": 8, "source": 3, "target": 2}, {"demand": 7, "source": 1, "target": 3}, {"demand": 7, "source": 3, "target": 1}, {"demand": 5, "source": 2, "target": 1}, {"demand": 8, "source": 2, "target": 3}]
Now with indentation
[
    {
        "demand": 5, 
        "source": 1, 
        "target": 2
    }, 
    {
        "demand": 8, 
        "source": 3, 
        "target": 2
    }, 
    {
        "demand": 7, 
        "source": 1, 
        "target": 3
    }, 
    {
        "demand": 7, 
        "source": 3, 
        "target": 1
    }, 
    {
        "demand": 5, 
        "source": 2, 
        "target": 1
    }, 
    {
        "demand": 8, 
        "source": 2, 
        "target": 3
    }
]


To read a JSON representation of demands and turn it into a demand dictionary we can reverse the process. Once again we'll use a helper function.

In [8]:
# Sample JSON demand string, we use a "\" to break a long string into multiple lines.
demand_string = '[{"source": 1, "target": 2, "demand": 5}, {"source": 3, "target": 2, "demand": 8},\
{"source": 1, "target": 3, "demand": 7}, {"source": 3, "target": 1, "demand": 7},\
{"source": 2, "target": 1, "demand": 5}, {"source": 2, "target": 3, "demand": 8}]'

In [9]:
# Try deserializing the above JSON string
p_demand = json.loads(demand_string)
print p_demand

[{u'source': 1, u'target': 2, u'demand': 5}, {u'source': 3, u'target': 2, u'demand': 8}, {u'source': 1, u'target': 3, u'demand': 7}, {u'source': 3, u'target': 1, u'demand': 7}, {u'source': 2, u'target': 1, u'demand': 5}, {u'source': 2, u'target': 3, u'demand': 8}]


In [10]:
def j_to_demands(d_list):
    """ Helps read in a demand dictionary from a JSON object.
        Assumes that d_list is a list of Python dictionaries representing the
        JSON demand and has from, to, and volume keys.
    """
    tmp = {}
    for d in d_list:
        tmp[d["source"], d["target"]] = d["demand"]
    return tmp

In [11]:
# Try deserializing the above JSON string
demands2 = j_to_demands(json.loads(demand_string))
print demands2
print demands2 == demands

{(1, 2): 5, (3, 2): 8, (1, 3): 7, (3, 1): 7, (2, 1): 5, (2, 3): 8}
True


When using a *link-path* formulation for network design problems we need a list of candidate paths. Furthermore it is very convenient to organize these by which demand pairs apply. We will use a node list representation for a path, and  have a list of candidate paths per demand pair.

For a simple three node network we could have a candidate path dictionary that looks something like:

In [12]:
paths = {(1, 2): [[1, 2], [1, 3, 2]],
 (1, 3): [[1, 3], [1, 2, 3]],
 (2, 1): [[2, 1]],
 (2, 3): [[2, 3]],
 (3, 1): [[3, 1], [3, 2, 1]],
 (3, 2): [[3, 2]]}

Note that Python tuples don't always need to be in parenthesis so we can get a list of paths for a demand with simple syntax such as:

In [13]:
print paths[1,3]

[[1, 3], [1, 2, 3]]


To convert to a nice JSON form we'll use a helper function:

In [14]:
def paths_to_j(path_dict):
    "Takes a candidate paths dictionary and converts it to a JSON serializable form."
    tmp = []  # We'll use a list
    for k in path_dict.keys(): 
        tmp.append({"source": k[0], "target": k[1], "paths": path_dict[k]})
    return tmp

In [15]:
print json.dumps(paths_to_j(paths))

[{"source": 1, "target": 2, "paths": [[1, 2], [1, 3, 2]]}, {"source": 3, "target": 2, "paths": [[3, 2]]}, {"source": 1, "target": 3, "paths": [[1, 3], [1, 2, 3]]}, {"source": 3, "target": 1, "paths": [[3, 1], [3, 2, 1]]}, {"source": 2, "target": 1, "paths": [[2, 1]]}, {"source": 2, "target": 3, "paths": [[2, 3]]}]


Now we'll create helper function to go from JSON to a candidate path dictionary.

In [16]:
def j_to_paths(path_list):
    """ A helper function to retreive a candidate path dictionary from JSON
        Assumes that path_list is a list of dictionaries, each representing a JSON path candidate
        object with from, to, and paths keywords.
    """
    tmp = {}
    for p in path_list:
        tmp[p["source"],p["target"]] = p["paths"]
    return tmp

In [17]:
# Example JSON string for candidate paths
path_string = '[{"target": 2, "source": 1, "paths": [[1, 2], [1, 3, 2]]}, \
{"target": 2, "source": 3, "paths": [[3, 2]]}, {"target": 3, "source": 1, "paths": [[1, 3], [1, 2, 3]]},\
{"target": 1, "source": 3, "paths": [[3, 1], [3, 2, 1]]}, {"target": 1, "source": 2, "paths": [[2, 1]]},\
{"target": 3, "source": 2, "paths": [[2, 3]]}]'

In [18]:
# Try converting from JSON path string back to a candidate path dictionary
paths2 = j_to_paths(json.loads(path_string))
print paths == paths2
print paths2

True
{(1, 2): [[1, 2], [1, 3, 2]], (3, 2): [[3, 2]], (1, 3): [[1, 3], [1, 2, 3]], (3, 1): [[3, 1], [3, 2, 1]], (2, 1): [[2, 1]], (2, 3): [[2, 3]]}
